In [ ]:
google_colab = True
if google_colab:
    from google.colab import drive

    drive.mount('/content/drive', force_remount=True)
    base_path = '/content/drive/MyDrive/MasterThesis/'
else:
    base_path = '../.'

Mounted at /content/drive


In [ ]:
if google_colab:
    !pip install socceraction matplotsoccer
    !pip install pandas
    !pip install tables
    !pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 86 kB 3.0 MB/s 
     |████████████████████████████████| 197 kB 44.8 MB/s 
     |████████████████████████████████| 6.4 MB 59.4 MB/s 
  Created wheel for matplotsoccer: filename=matplotsoccer-0.0.8-py3-none-any.whl size=5983 sha256=83fa1ca8f8462ea3400f02201f99254adbc0565da1603d2c4bf39fdd3b6ea5a7
  Stored in directory: /root/.cache/pip/wheels/be/6f/64/335f5a5e7763b284f2f7d8893f20338de4d9f73794fe85ea7d
Successfully built matplotsoccer
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [ ]:

import os
import json
import warnings
import shutil
import json
import pandas as pd
from matplotlib import pyplot as plt
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from enum import Enum

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

from socceraction.data.statsbomb import StatsBombLoader
import matplotsoccer as mps
from socceraction.vaep import VAEP
import socceraction.spadl as spadl
import socceraction.vaep.features as fs
import socceraction.vaep.labels as labs
from socceraction.data.statsbomb import StatsBombLoader
import matplotsoccer as mps
from socceraction.vaep import VAEP
import socceraction.spadl as spadl
import socceraction.vaep.features as fs
import socceraction.vaep.labels as labs
import socceraction.vaep.formula as vaepformula
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch import nn
from sklearn.metrics import brier_score_loss, roc_auc_score, log_loss
from sklearn.metrics import roc_curve, confusion_matrix, precision_recall_curve

import plotly.graph_objects as go
import plotly.figure_factory as ff

if google_colab:
    % load_ext autoreload
    % autoreload 2
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))


class dotdict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

1
Tesla P100-PCIE-16GB


In [ ]:
if google_colab:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = "cpu"
seed = 42


def seed_new_generator():
    return torch.Generator().manual_seed(seed)


os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True
    print(torch.cuda.current_device())

0


In [ ]:
if google_colab:
    os.system(f'git clone https://github.com/statsbomb/open-data {base_path}')

In [ ]:
if google_colab:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)

    from psutil import virtual_memory

    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Thu Jul 21 20:55:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class StatsBombDataset(Enum):
    BARCELONA = 1
    WOMEN = 2

In [ ]:
dataset_version = StatsBombDataset.WOMEN

if dataset_version == StatsBombDataset.BARCELONA:
    datafolder = os.path.join(base_path, "data-barcelona/")
elif dataset_version == StatsBombDataset.WOMEN:
    datafolder = os.path.join(base_path, "data-women/")

archive_path = "./"

spadl_h5 = os.path.join(datafolder, "spadl-statsbomb.h5")
print(os.path.exists(spadl_h5))
features_h5 = os.path.join(datafolder, "features.h5")
labels_h5 = os.path.join(datafolder, "labels.h5")
predictions_h5 = os.path.join(datafolder, "predictions.h5")
vaep_values_h5 = os.path.join(datafolder, "vaep_values.h5")

neural_features_h5 = os.path.join(datafolder, "neural_features.h5")
neural_predictions_h5 = os.path.join(datafolder, "neural_predictions.h5")
substitutions_h5 = os.path.join(datafolder, "substitutions.h5")

player_features_h5 = os.path.join(datafolder, "player_features.h5")
player_substitutions_h5 = os.path.join(datafolder, "player_substitutions.h5")
player_list_per_season_json = os.path.join(datafolder, "player_list_per_season.json")
player_list_name_per_season_json = os.path.join(datafolder, "player_list_name_per_season.json")
cumsum_current_player_stats_npy = os.path.join(datafolder, "cumsum_current_player_stats.npy")
current_player_stats_npy = os.path.join(datafolder, "cumulated_player_stats.npy")
context_game_stats_npy = os.path.join(datafolder, "context_game_stats.npy")
cumsum_history_player_stats_npy = os.path.join(datafolder, "cumsum_history_player_stats.npy")
history_player_stats_npy = os.path.join(datafolder, "history_player_stats.npy")
player_lineups_npy = os.path.join(datafolder, "player_lineups")
game_substitutions_npy = os.path.join(datafolder, "game_substitutions")
home_player_lineups_npy = os.path.join(datafolder, "home_player_lineups")
away_player_lineups_npy = os.path.join(datafolder, "away_player_lineups")
home_game_substitutions_npy = os.path.join(datafolder, "home_game_substitutions")
away_game_substitutions_npy = os.path.join(datafolder, "away_game_substitutions")

SBL = StatsBombLoader(root=os.path.join(base_path, 'data'), getter='local')

assert os.path.exists(datafolder)
# Create data folder if it doesn't exist
if not os.path.exists(datafolder):
    os.mkdir(datafolder)

True


In [ ]:
if os.path.exists(spadl_h5):
    games = pd.read_hdf(spadl_h5, "games")
else:
    ## Change to select competitions
    competitions = SBL.competitions()
    if dataset_version == StatsBombDataset.BARCELONA:
        selected_competitions = competitions[competitions.competition_name == 'La Liga']
    elif dataset_version == StatsBombDataset.WOMEN:
        selected_competitions = competitions[competitions.competition_name == "FA Women's Super League"]
    games = pd.concat([
        SBL.games(row.competition_id, row.season_id)
        for row in selected_competitions.itertuples()
    ]).reset_index()

    ## Change to select games
    # games = games.head(1)
    teams, players = [], []
    actions = {}

    for game in tqdm(list(games.itertuples()), desc="Loading game data"):
        teams.append(SBL.teams(game.game_id))
        players.append(SBL.players(game.game_id))
        events = SBL.events(game.game_id)
        actions[game.game_id] = spadl.statsbomb.convert_to_actions(events, game.home_team_id)

    teams = pd.concat(teams).drop_duplicates(subset="team_id")
    players = pd.concat(players)

    with pd.HDFStore(spadl_h5) as store:
        store["competitions"] = selected_competitions
        store["games"] = games
        store["teams"] = teams
        store["players"] = players[['player_id', 'player_name', 'nickname']].drop_duplicates(subset='player_id')
        store["player_games"] = players[
            ['player_id', 'game_id', 'team_id', 'is_starter', 'starting_position_id', 'starting_position_name',
             'minutes_played']]
        for game_id in actions.keys():
            store[f'actions/game_{game_id}'] = actions[game_id]

        f_games = (store["games"]
                   .merge(store["competitions"], how='left')
                   .merge(store["teams"].add_prefix('home_'), how='left')
                   .merge(store["teams"].add_prefix('away_'), how='left')
                   )

        game_id = f_games.game_id.values[0]
        f_actions = (
            store[f"actions/game_{game_id}"]
                .merge(spadl.actiontypes_df(), how='left')
                .merge(spadl.results_df(), how='left')
                .merge(spadl.bodyparts_df(), how='left')
                .merge(store["players"], how='left')
                .merge(store["teams"], how='left')
        )

        f_actions["player_name"] = f_actions[["nickname", "player_name"]].apply(lambda x: x[0] if x[0] else x[1],
                                                                                axis=1)
        del f_actions['nickname']


In [ ]:
nb_actions_before = 10
xfns = [
    fs.actiontype,
    fs.actiontype_onehot,
    fs.bodypart,
    fs.bodypart_onehot,
    fs.result,
    fs.result_onehot,
    fs.goalscore,
    fs.startlocation,
    fs.endlocation,
    fs.movement,
    fs.space_delta,
    fs.startpolar,
    fs.endpolar,
    fs.team,
    fs.time,
    fs.time_delta
]
yfns = [labs.scores,
        labs.concedes,
        labs.goal_from_shot]

if not os.path.exists(features_h5):
    for game in tqdm(games.itertuples(), desc=f"Generating and storing features in {features_h5}"):
        actions = pd.read_hdf(spadl_h5, f"actions/game_{game.game_id}")
        gamestates = fs.gamestates(spadl.add_names(actions), nb_actions_before)
        gamestates = fs.play_left_to_right(gamestates, game.home_team_id)

        X = pd.concat([fn(gamestates) for fn in xfns], axis=1)
        X.to_hdf(features_h5, f"game_{game.game_id}")

if not os.path.exists(labels_h5):
    for game in tqdm(games.itertuples(), desc=f"Generating and storing features in {labels_h5}"):
        actions = pd.read_hdf(spadl_h5, f"actions/game_{game.game_id}")
        Y = pd.concat([fn(spadl.add_names(actions)) for fn in yfns], axis=1)
        Y.to_hdf(labels_h5, f"game_{game.game_id}")


In [ ]:
import xgboost
from sklearn.metrics import brier_score_loss, roc_auc_score, log_loss

games = games.sample(frac=1)
nb_games = len(games.index)
train_games = games
test_games = games
## Feature set
xfns = [
    fs.actiontype,
    fs.actiontype_onehot,
    fs.bodypart,
    fs.bodypart_onehot,
    fs.result,
    fs.result_onehot,
    fs.goalscore,
    fs.startlocation,
    fs.endlocation,
    fs.movement,
    fs.space_delta,
    fs.startpolar,
    fs.endpolar,
    fs.team,
    fs.time,
    fs.time_delta,
    # fs.actiontype_result_onehot
]


def getXY(games, Xcols):
    X = []
    for game_id in tqdm(games.game_id, desc="Selecting features"):
        Xi = pd.read_hdf(features_h5, f"game_{game_id}")
        X.append(Xi[Xcols])
    X = pd.concat(X).reset_index(drop=True)

    Y = []
    for game_id in tqdm(games.game_id, desc="Selecting labels"):
        Yi = pd.read_hdf(labels_h5, f"game_{game_id}")
        Y.append(Yi[Ycols])
    Y = pd.concat(Y).reset_index(drop=True)
    return X, Y


def evaluate(y, y_hat):
    print(f'Number of datapoints : {len(y)}')
    p = sum(y) / len(y)
    base = [p] * len(y)
    brier = brier_score_loss(y, y_hat)
    print(f"  Brier score: %.5f (BSS %.5f)" % (brier, 1 - (brier / brier_score_loss(y, base))))
    ll = log_loss(y, y_hat)
    print(f"  log loss score: %.5f (%.5f)" % (ll, ll / log_loss(y, base)))
    print(f"  ROC AUC: %.5f" % roc_auc_score(y, y_hat))


def save_predictions(model):
    A = []
    for game_id in tqdm(test_games.game_id, "Loading game ids"):
        Ai = pd.read_hdf(spadl_h5, f"actions/game_{game_id}")
        A.append(Ai[["game_id"]])

    A = pd.concat(A)
    A = A.reset_index(drop=True)
    grouped_predictions = pd.concat([A, Y_hat], axis=1).groupby('game_id')
    for k, df in tqdm(grouped_predictions, desc="Saving predictions per game"):
        df = df.reset_index(drop=True)
        df[Y_hat.columns].to_hdf(predictions_h5, predict_path + f"game_{int(k)}")


def getXYRNN(games):
    X = []
    for game_id in tqdm(games.head(1).game_id, desc="Selecting features"):
        actions = pd.read_hdf(spadl_h5, f"actions/game_{game_id}")
        print(actions)


def evaluate_baseline(games):
    Y_hat = []
    Y = []
    for game in tqdm(games.itertuples(), desc="Evaluating baseline"):
        game_labels = pd.read_hdf(labels_h5, f'game_{game.game_id}')
        Y_hat.append(game_labels[["baseline_substitution"]])
        Y.append(game_labels[["substitution"]])
    Y_hat = pd.concat(Y_hat).reset_index(drop=True)['baseline_substitution'].apply(lambda x: 1 if x == 100 else 0)
    Y = pd.concat(Y).reset_index(drop=True)['substitution'].apply(lambda x: 1 if x else 0)
    Y_hat_baseline = np.full(len(Y), Y_hat.value_counts()[1] / Y_hat.value_counts()[0])

    evaluate(Y, Y_hat)
    evaluate(Y, Y_hat_baseline)


if not os.path.exists(predictions_h5):
    Xcols = fs.feature_column_names(xfns, nb_actions_before)
    Ycols = ["scores", "concedes"]
    predict_path = 'goals/'

    Xtrain, Ytrain = getXY(train_games, Xcols)
    Xtest, Ytest = Xtrain, Ytrain
    Y_hat = pd.DataFrame()
    models = {}
    for col in Ytrain.columns:
        if google_colab:
            model = xgboost.XGBClassifier(n_estimators=50, max_depth=3, n_jobs=-3, verbosity=1, tree_method='gpu_hist')
        else:
            model = xgboost.XGBClassifier(n_estimators=50, max_depth=3, n_jobs=-3, verbosity=1)
        model.fit(Xtrain, Ytrain[col])
        models[col] = model
    for game_id in tqdm(games.game_id, "Loading features to "):
        Xi = pd.read_hdf(features_h5, f"game_{game_id}")
        Xi = Xi[Xcols].reset_index(drop=True)
        Yi = {"scores": [], "concedes": []}
        for col in Ycols:
            Yi[col] = models[col].predict_proba(Xi)[:, 1]
        Yi = pd.DataFrame(Yi, index=Xi.index)
        Yi.to_hdf(predictions_h5, f"goals/game_{game_id}")

    # evaluate_baseline(games)

    #Xtest, Ytest = Xtrain, Ytrain
    #Y_hat = train_model_XGBC()
    #save_predictions(Y_hat)


In [ ]:
def fix_event_game_time_seconds(events):
    events['time_seconds'] = events.minute * 60 + events.second
    first_half_events = events[events.period_id == 1]
    first_half_time = first_half_events.iloc[0].time_seconds
    second_half_events_mask = events.period_id == 2
    events.loc[second_half_events_mask, 'time_seconds'] += first_half_time
    return events


def fix_spadl_actions_time_seconds(actions):
    first_half_actions = actions[actions.period_id == 1]
    rev_first_half_actions = first_half_actions.sort_index(axis=0, ascending=False)
    first_half_time = rev_first_half_actions.iloc[0].time_seconds
    second_half_actions_mask = actions.period_id == 2
    actions.loc[second_half_actions_mask, 'time_seconds'] += first_half_time
    return actions


def substitution_summary(substitutions):
    sub_copy = substitutions
    group_by_minute = sub_copy.groupby(['minute'])['time_seconds'].count()
    sub_before_45 = group_by_minute.loc[0:44].sum()
    sub_at_45 = group_by_minute.loc[45].sum()
    sub_after_45 = group_by_minute.loc[46:].sum()

    group_by_game = sub_copy.groupby(['game_id']).count()
    group_by_game_minute = sub_copy.groupby(['game_id', 'minute'])['time_seconds'].count()
    nb_times_more_than_1_sub_minute = group_by_game_minute.value_counts()
    print(f'Total number of substitutions : {len(substitutions)}\n ...')


def extract_substitutions_from_games(games):
    substitution_event_id = 19
    substitutions = []

    for game in tqdm(games.itertuples(), desc='Finding substitution times'):
        events = SBL.events(game.game_id)
        events = fix_event_game_time_seconds(events)
        events.to_hdf(spadl_h5, f'events/game_{game.game_id}')
        game_sub = events[events.type_id == substitution_event_id]
        substitutions.append(game_sub)
    substitutions = pd.concat(substitutions, axis=0, ignore_index=True).reset_index()
    substitutions.to_hdf(substitutions_h5, 'substitutions')
    return substitutions


def encode_substitutions_in_labels(games, substitutions):
    nb_minutes_before_sub = 3
    for game in tqdm(games.itertuples(), desc="Encoding substitutions in game's labels"):
        actions = pd.read_hdf(spadl_h5, f'actions/game_{game.game_id}')
        actions = fix_spadl_actions_time_seconds(actions)
        game_sub = substitutions[substitutions.game_id == game.game_id]
        game_labels = pd.read_hdf(labels_h5, f'game_{game.game_id}')
        game_labels['substitution'] = False
        for team_id in [game.home_team_id, game.away_team_id]:
            team_action_mask = actions.team_id == team_id
            team_actions = actions.loc[team_action_mask]
            team_subs = game_sub[game_sub.team_id == team_id]
            if not team_subs.empty:
                for sub in team_subs.itertuples():
                    sub_time = sub.time_seconds
                    actions_within_sub_window_mask = (actions.team_id == team_id) & (
                            actions.time_seconds < sub_time) & (
                                                             actions.time_seconds > sub_time - nb_minutes_before_sub * 60)
                    sub_actions = actions[actions_within_sub_window_mask]
                    if sub.minute == 45:
                        game_labels.loc[actions_within_sub_window_mask,
                                        'substitution'] = True
                    else:
                        if not sub_actions.empty:
                            last_sub_action_index = sub_actions.index[-1]
                            game_labels.loc[actions_within_sub_window_mask,
                                            'substitution'] = True
        game_labels.to_hdf(labels_h5, f'game_{game.game_id}')


if not os.path.exists(substitutions_h5):
    games = pd.read_hdf(spadl_h5, "games")
    print("nb of games:", len(games))
    substitutions = extract_substitutions_from_games(games)
    encode_substitutions_in_labels(games, substitutions)




In [ ]:
def compute_vaep_values(games):
    for game in tqdm(games.itertuples(), desc='Computing VAEP values'):
        predictions = pd.read_hdf(predictions_h5, f"goals/game_{game.game_id}")
        actions = pd.read_hdf(spadl_h5, f"actions/game_{game.game_id}")
        players = pd.read_hdf(spadl_h5, "players")
        teams = pd.read_hdf(spadl_h5, "teams")
        actions = (
            spadl.add_names(actions)
                .merge(players, how="left")
                .merge(teams, how="left")
                .sort_values(["game_id", "period_id", "action_id"])
                .reset_index(drop=True)
        )
        vaep_values = vaepformula.value(actions, predictions.scores, predictions.concedes)
        vaep_values.to_hdf(vaep_values_h5, f'game_{game.game_id}')


if not os.path.exists(vaep_values_h5):
    compute_vaep_values(games)

In [ ]:
from sklearn.preprocessing import LabelEncoder


def normalize_columns(df, columns):
    df_min_max_scaled = df.copy()
    for column in columns:
        df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (
                df_min_max_scaled[column].max() - df_min_max_scaled[column].min())
    return df_min_max_scaled


def from_bool_to_integer_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: 1 if x else 0)
    return df


def encode_team_ids(df, column_name):
    list_team_ids = df[column_name].unique()
    df[column_name] = df[column_name].astype('category').cat.codes
    return df


def extract_features(games):
    neural_features = []
    for game in tqdm(games.itertuples(), "Extracting features for neural network"):
        game_actions = pd.read_hdf(spadl_h5, f'actions/game_{game.game_id}')
        vaep_values = pd.read_hdf(vaep_values_h5, f'game_{game.game_id}')['vaep_value']
        game_labels = pd.read_hdf(labels_h5, f'game_{game.game_id}')
        features = pd.read_hdf(features_h5, f'game_{game.game_id}')
        extracted_real_features = features[features_real_columns]
        extracted_discrete_features = features[features_discrete_columns]
        extracted_time_features = features[time_column]
        extracted_actions = game_actions[actions_columns]
        extracted_labels = game_labels['substitution']
        game_features = pd.concat(
            [extracted_real_features,
             vaep_values,
             extracted_discrete_features, extracted_actions,
             extracted_time_features,
             extracted_labels],
            axis=1)
        game_features['goalscore_diff'] = game_features['goalscore_diff']
        game_features['period_id_a0'] = game_features['period_id_a0'] - 1
        game_features['game_id'] = game.game_id
        neural_features.append(game_features)
    return pd.concat(neural_features, axis=0).reset_index(drop=True)


def create_neural_features(games):
    raw_neural_features = extract_features(games)
    normalized_features = normalize_columns(raw_neural_features, features_real_columns)
    formatted_labels_features = from_bool_to_integer_columns(normalized_features, ['substitution'])
    finalized_features = encode_team_ids(formatted_labels_features, 'team_id')
    return finalized_features


def save_neural_features_for_each_games(neural_features, games):
    games.to_hdf(neural_features_h5, 'game_list')
    neural_features.to_hdf(neural_features_h5, 'all_neural_features')
    for game in tqdm(games.itertuples(), "Saving neural features for each games"):
        game_data = neural_features[neural_features['game_id'] == game.game_id].reset_index(drop=True)
        game_data.to_hdf(neural_features_h5, f'neural_features/game_{game.game_id}')


features_real_columns = ['start_x_a0',
                         'start_y_a0',
                         'end_x_a0',
                         'end_y_a0',
                         'dx_a0',
                         'dy_a0',
                         'movement_a0',
                         'start_dist_to_goal_a0',
                         'start_angle_to_goal_a0',
                         'end_dist_to_goal_a0',
                         'end_angle_to_goal_a0',
                         ]
features_discrete_columns = [
    'goalscore_diff',
    'goalscore_team',
    'goalscore_opponent',
    'period_id_a0',
]
actions_columns = ['type_id',
                   'result_id',
                   'bodypart_id',
                   'team_id']
time_column = 'time_seconds_overall_a0'

label_encoders = {}
#Count the number of different discrete features to prepare Embedded layers
discrete_features_names = np.concatenate((features_discrete_columns, actions_columns))
discrete_features_value_counts = {}
features_column_index = {}
real_features_names = np.concatenate((features_real_columns, ['vaep_value']))

if not os.path.exists(neural_features_h5):
    neural_features = create_neural_features(games)
else:
    neural_features = pd.read_hdf(neural_features_h5, 'all_neural_features')

for feature_name in discrete_features_names:
    lb_make = LabelEncoder()
    if not os.path.exists(neural_features_h5):
        neural_features[feature_name] = lb_make.fit_transform(neural_features[feature_name])
    else:
        neural_features[feature_name] = lb_make.fit(neural_features[feature_name])
    label_encoders[feature_name] = lb_make
    discrete_features_value_counts[feature_name] = len(lb_make.classes_)
    features_column_index[feature_name] = neural_features.columns.get_loc(feature_name)
for feature_name in real_features_names:
    features_column_index[feature_name] = neural_features.columns.get_loc(feature_name)
features_column_index[time_column] = neural_features.columns.get_loc(time_column)
if not os.path.exists(neural_features_h5):
    save_neural_features_for_each_games(neural_features, games)
del neural_features

In [ ]:
max_minutes = 0
player_list_per_season = {}
barcelona_id = 217

seasons = pd.read_hdf(spadl_h5, "competitions")
sorted_seasons = seasons.sort_values('season_name', ascending=True)
game_id_list = []
for season in seasons.itertuples():
    season_games = games[games.season_id == season.season_id]
    sorted_season_games = season_games.sort_values('game_day', ascending=True)
    for game in sorted_season_games.itertuples():
        game_id_list.append(int(game.game_id))

In [ ]:
def extract_lineups(games):
    for game in tqdm(games.itertuples(), desc=f'Extracting team lineups'):
        players = []
        with open(os.path.join(base_path, f'data/lineups/{game.game_id}.json'), 'r') as jsonfile:
            lineup = json.load(jsonfile)
            for team_obj in lineup:
                for player in team_obj['lineup']:
                    players.append(
                        {'team_id': team_obj['team_id'], 'player_id': player['player_id'],
                         'player_name': player['player_name'] if player['player_nickname'] == None else player[
                             'player_nickname']})
        players = pd.DataFrame(players)
        players.to_hdf(player_features_h5, f'lineups/game_{game.game_id}')


if not os.path.exists(player_features_h5):
    extract_lineups(games)

In [ ]:
def get_season_teams(season_games):
    unique_home_team_ids = pd.unique(season_games['home_team_id'])
    unique_away_team_ids = pd.unique(season_games['away_team_id'])
    return np.union1d(unique_home_team_ids, unique_away_team_ids)


if not os.path.exists(player_list_per_season_json):
    player_list_per_season = dotdict(dict())
    for season in seasons.itertuples():
        player_list_per_season[str(season.season_id)] = dotdict(dict())
        season_games = games[games.season_id == season.season_id]
        if dataset_version == StatsBombDataset.BARCELONA:
            season_players = pd.DataFrame(columns=['player_id', 'player_name'])
        else:
            season_teams = get_season_teams(season_games)
            for team in season_teams:
                player_list_per_season[str(season.season_id)][str(team)] = pd.DataFrame(
                    columns=['player_id', 'player_name'])
        sorted_season_games = season_games.sort_values('game_day', ascending=True)
        for game in tqdm(sorted_season_games.itertuples()):
            players = pd.read_hdf(player_features_h5, f'lineups/game_{game.game_id}')
            if dataset_version == StatsBombDataset.BARCELONA:
                barca_players = players[players['team_id'] == barcelona_id]
                season_players = pd.concat(
                    [season_players, barca_players[['player_id', 'player_name']]]).drop_duplicates().reset_index(
                    drop=True)
            else:
                home_players = players[players['team_id'] == game.home_team_id]
                away_players = players[players['team_id'] == game.away_team_id]
                player_list_per_season[str(season.season_id)][str(game.home_team_id)] = pd.concat(
                    [player_list_per_season[str(season.season_id)][str(game.home_team_id)],
                     home_players[['player_id', 'player_name']]]).drop_duplicates().reset_index(drop=True)
                player_list_per_season[str(season.season_id)][str(game.away_team_id)] = pd.concat(
                    [player_list_per_season[str(season.season_id)][str(game.away_team_id)],
                     away_players[['player_id', 'player_name']]]).drop_duplicates().reset_index(drop=True)
            min_game = max(SBL.events(game.game_id)['minute'].unique())
            if max_minutes < min_game: max_minutes = min_game
        if dataset_version == StatsBombDataset.BARCELONA:
            player_list_per_season[str(season.season_id)] = season_players.to_json()
        else:
            for team in season_teams:
                player_list_per_season[str(season.season_id)][str(team)] = player_list_per_season[str(season.season_id)][str(team)].to_json()

    player_list_per_season['max_minutes'] = int(max_minutes)
    with open(player_list_per_season_json, "w") as fp:
        json.dump(player_list_per_season, fp)
else:
    with open(player_list_per_season_json, 'r') as fp:
        player_list_per_season = json.load(fp)
        max_minutes = player_list_per_season['max_minutes']

In [ ]:
from sklearn.preprocessing import LabelEncoder

player_id_encoder = LabelEncoder()

player_list = pd.DataFrame(columns=['player_id', 'player_name'])
for season in seasons.itertuples():
    if dataset_version == StatsBombDataset.BARCELONA:
        season_players = pd.read_json(player_list_per_season[str(season.season_id)])
    else:
        season_players = pd.DataFrame(columns=['player_id', 'player_name'])
        for team_id in player_list_per_season[str(season.season_id)].keys():
            team_players = pd.read_json(player_list_per_season[str(season.season_id)][team_id])
            season_players = pd.concat([season_players, team_players]).drop_duplicates().reset_index(drop=True)
    player_list = pd.concat([player_list, season_players]).drop_duplicates().reset_index(drop=True)

full_player_list = np.array(list(player_list['player_id'])).astype(int)
full_player_name_list = np.array(list(player_list['player_name']))
encoded_player_ids = player_id_encoder.fit_transform(full_player_list)
player_id_to_index = dict(zip(full_player_list, encoded_player_ids))
id_dict = dict(zip(encoded_player_ids, full_player_list))
name_dict = dict(zip(encoded_player_ids, full_player_name_list))

In [ ]:
if not os.path.exists(cumsum_current_player_stats_npy):
    vaep_values_per_player = []
    shot_action_type_id = 11
    pass_action_type_id = 0
    carry_action_type_id = 21
    shot_event_type_id = 16
    pass_event_type_id = 30
    carry_event_type_id = 43

    player_stats_num_features = 6
    context_stats_game_num_features = 9

    goal_difference_stats = np.zeros((len(games), max_minutes + 1, 1))
    if dataset_version == StatsBombDataset.BARCELONA:
        player_stats_game = np.zeros((len(games), max_minutes + 1, len(encoded_player_ids), player_stats_num_features))
        context_stats_game = np.zeros((len(games), max_minutes + 1, context_stats_game_num_features))
    else:
        player_stats_game = np.zeros((len(games) * 2, max_minutes + 1, len(encoded_player_ids), player_stats_num_features))
        context_stats_game = np.zeros((len(games) * 2, max_minutes + 1, context_stats_game_num_features))

    game_i = 0
    season_i = 0
    for season in seasons.itertuples():
        season_games = games[games.season_id == season.season_id]
        sorted_season_games = season_games.sort_values('game_day', ascending=True)
        for game in tqdm(list(sorted_season_games.itertuples())):
            actions = pd.read_hdf(spadl_h5, f'actions/game_{game.game_id}')
            vaep = pd.read_hdf(vaep_values_h5, f'game_{game.game_id}')
            events = SBL.events(game.game_id)
            neural_features = pd.read_hdf(neural_features_h5, f'neural_features/game_{game.game_id}')
            neural_features = neural_features.drop(['team_id', 'type_id', 'game_id', 'vaep_value'], axis=1)
            actions['vaep_value'] = vaep['vaep_value']
            actions = pd.concat([actions, neural_features], axis=1)
            actions = actions.merge(events[['event_id', 'minute']], left_on='original_event_id', right_on='event_id')
            actions['gap_difference'] = actions['end_dist_to_goal_a0'] - actions['start_dist_to_goal_a0']
            actions['goal_difference'] = actions['goalscore_team'] - actions['goalscore_opponent']
            if dataset_version == StatsBombDataset.BARCELONA:
                team_actions = actions[actions['team_id'] == barcelona_id]
                grouped_actions = team_actions.groupby(['minute', 'player_id'])
                grouped_action_types = team_actions.groupby(['minute', 'player_id', 'type_id'])
                for (min, player_id), player_actions in grouped_actions:
                    player_stats_game[game_i][min][player_id_to_index[player_id]][0] = player_actions.sum()[
                        'vaep_value']
                    player_stats_game[game_i][min][player_id_to_index[player_id]][4] = player_actions.sum()[
                        'movement_a0']
                    player_stats_game[game_i][min][player_id_to_index[player_id]][5] = player_actions.sum()[
                        'gap_difference']
                for (min, player_id, type_id), player_actions in grouped_action_types:
                    if type_id == shot_action_type_id:
                        player_stats_game[game_i][min][player_id_to_index[player_id]][1] = len(player_actions.index)
                    elif type_id == pass_action_type_id:
                        player_stats_game[game_i][min][player_id_to_index[player_id]][2] = len(player_actions.index)
                    elif type_id == carry_action_type_id:
                        player_stats_game[game_i][min][player_id_to_index[player_id]][3] = len(player_actions.index)
                context_actions = actions[actions['team_id'] != barcelona_id]
                grouped_context_actions = context_actions.groupby(['minute'])
                grouped_context_action_types = context_actions.groupby(['minute', 'type_id'])
                for min, actions in grouped_context_actions:
                    context_stats_game[game_i][min][0] = actions.sum()['vaep_value']
                    context_stats_game[game_i][min][4] = actions.sum()['movement_a0']
                    context_stats_game[game_i][min][5] = actions.sum()['gap_difference']
                    context_stats_game[game_i][min][6] = actions.iloc[-1]['goal_difference']
                    context_stats_game[game_i][min][7] = actions.iloc[-1]['goalscore_team']
                    context_stats_game[game_i][min][8] = actions.iloc[-1]['goalscore_opponent']
                for (min, type_id), actions in grouped_context_action_types:
                    if type_id == shot_action_type_id:
                        context_stats_game[game_i][min][1] = len(actions.index)
                    elif type_id == pass_action_type_id:
                        context_stats_game[game_i][min][2] = len(actions.index)
                    elif type_id == carry_action_type_id:
                        context_stats_game[game_i][min][3] = len(actions.index)
            else:
                home_actions = actions[actions['team_id'] == game.home_team_id]
                away_actions = actions[actions['team_id'] == game.away_team_id]

                # minutes = team_actions['minute'].unique()

                grouped_home_actions = home_actions.groupby(['minute', 'player_id'])
                grouped_away_actions = away_actions.groupby(['minute', 'player_id'])
                grouped_home_action_types = home_actions.groupby(['minute', 'player_id', 'type_id'])
                grouped_away_action_types = away_actions.groupby(['minute', 'player_id', 'type_id'])

                for (min, player_id), player_actions in grouped_home_actions:
                    player_stats_game[game_i][min][player_id_to_index[player_id]][0] = player_actions.sum()[
                        'vaep_value']
                    player_stats_game[game_i][min][player_id_to_index[player_id]][4] = player_actions.sum()[
                        'movement_a0']
                    player_stats_game[game_i][min][player_id_to_index[player_id]][5] = player_actions.sum()[
                        'gap_difference']
                for (min, player_id, type_id), player_actions in grouped_home_action_types:
                    if type_id == shot_action_type_id:
                        player_stats_game[game_i][min][player_id_to_index[player_id]][1] = len(player_actions.index)
                    elif type_id == pass_action_type_id:
                        player_stats_game[game_i][min][player_id_to_index[player_id]][2] = len(player_actions.index)
                    elif type_id == carry_action_type_id:
                        player_stats_game[game_i][min][player_id_to_index[player_id]][3] = len(player_actions.index)

                for (min, player_id), player_actions in grouped_away_actions:
                    player_stats_game[len(games) + game_i][min][player_id_to_index[player_id]][0] = player_actions.sum()['vaep_value']
                    player_stats_game[len(games) + game_i][min][player_id_to_index[player_id]][4] = player_actions.sum()[
                        'movement_a0']
                    player_stats_game[len(games) + game_i][min][player_id_to_index[player_id]][5] = player_actions.sum()[
                        'gap_difference']
                for (min, player_id, type_id), player_actions in grouped_away_action_types:
                    if type_id == shot_action_type_id:
                        player_stats_game[len(games) + game_i][min][player_id_to_index[player_id]][1] = len(
                            player_actions.index)
                    elif type_id == pass_action_type_id:
                        player_stats_game[len(games) + game_i][min][player_id_to_index[player_id]][2] = len(
                            player_actions.index)
                    elif type_id == carry_action_type_id:
                        player_stats_game[len(games) + game_i][min][player_id_to_index[player_id]][3] = len(
                            player_actions.index)

                home_context_actions = away_actions
                away_context_actions = home_actions

                grouped_home_context_actions = home_context_actions.groupby(['minute'])
                grouped_away_context_actions = away_context_actions.groupby(['minute'])
                grouped_home_context_action_types = home_context_actions.groupby(['minute', 'type_id'])
                grouped_away_context_action_types = away_context_actions.groupby(['minute', 'type_id'])

                for min, actions in grouped_home_context_actions:
                    context_stats_game[game_i][min][0] = actions.sum()['vaep_value']
                    context_stats_game[game_i][min][4] = actions.sum()['movement_a0']
                    context_stats_game[game_i][min][5] = actions.sum()['gap_difference']
                    context_stats_game[game_i][min][6] = actions.iloc[-1]['goal_difference']
                    context_stats_game[game_i][min][7] = actions.iloc[-1]['goalscore_team']
                    context_stats_game[game_i][min][8] = actions.iloc[-1]['goalscore_opponent']
                for (min, type_id), actions in grouped_home_context_action_types:
                    if type_id == shot_action_type_id:
                        context_stats_game[game_i][min][1] = len(actions.index)
                    elif type_id == pass_action_type_id:
                        context_stats_game[game_i][min][2] = len(actions.index)
                    elif type_id == carry_action_type_id:
                        context_stats_game[game_i][min][3] = len(actions.index)

                for min, actions in grouped_away_context_actions:
                    context_stats_game[len(games) + game_i][min][0] = actions.sum()['vaep_value']
                    context_stats_game[len(games) + game_i][min][4] = actions.sum()['movement_a0']
                    context_stats_game[len(games) + game_i][min][5] = actions.sum()['gap_difference']
                    context_stats_game[len(games) + game_i][min][6] = - actions.iloc[-1]['goal_difference']
                    context_stats_game[len(games) + game_i][min][7] = actions.iloc[-1]['goalscore_opponent']
                    context_stats_game[len(games) + game_i][min][8] = actions.iloc[-1]['goalscore_team']
                for (min, type_id), actions in grouped_away_context_action_types:
                    if type_id == shot_action_type_id:
                        context_stats_game[len(games) + game_i][min][1] = len(actions.index)
                    elif type_id == pass_action_type_id:
                        context_stats_game[len(games) + game_i][min][2] = len(actions.index)
                    elif type_id == carry_action_type_id:
                        context_stats_game[len(games) + game_i][min][3] = len(actions.index)

            game_i += 1
        season_i += 1
    current_player_stats = player_stats_game
    cumsum_current_player_stats = np.cumsum(current_player_stats, axis=1)

    with open(current_player_stats_npy, 'wb') as fp:
        np.save(fp, current_player_stats)
    with open(cumsum_current_player_stats_npy, 'wb') as fp:
        np.save(fp, cumsum_current_player_stats)
    with open(context_game_stats_npy, 'wb') as fp:
        np.save(fp, context_stats_game)

In [ ]:
if not os.path.exists(cumsum_history_player_stats_npy):
    i = 0
    history_player_stats = current_player_stats.copy()
    for season in seasons.itertuples():
        season_games = games[games.season_id == season.season_id]
        sorted_season_games = season_games.sort_values('game_day', ascending=True)

        season_first_game_index = i
        season_last_game_index = i + len(sorted_season_games)

        cumulated_player_stats_season_games = current_player_stats[season_first_game_index:season_last_game_index]

        for j in range(season_last_game_index - season_first_game_index):
            cumulated_before = cumulated_player_stats_season_games[0:j + 1]
            expected_before = np.sum(cumulated_before, axis=0) / (j + 1)
            history_player_stats[season_first_game_index + j] = expected_before

        i = season_last_game_index

    cumsum_history_player_stats = np.cumsum(history_player_stats, axis=1)

    with open(history_player_stats_npy, 'wb') as fp:
        np.save(fp, history_player_stats)

    with open(cumsum_history_player_stats_npy, 'wb') as fp:
        np.save(fp, cumsum_history_player_stats)

In [ ]:
def get_substitution_batches(sub_events, nb_sub_batches):
    subs = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
    for i in range(nb_sub_batches):
        if len(sub_events) > 0:
            next_sub_event = sub_events.iloc[0]

            minute, second = next_sub_event['minute'], next_sub_event['second']
            same_time_frame_events = sub_events[
                (sub_events.minute == minute) | ((sub_events.minute == minute + 1) & (sub_events.second <= second))]
            sub_events = sub_events.loc[sub_events.index.difference(same_time_frame_events.index)]
            subs[i] = same_time_frame_events
    return subs

In [ ]:
starting_XI_type_id = 35
player_on_bench_index = 0
player_on_field_index = 1
player_in_index = 0
player_out_index = 1
nb_sub_batches = 3

if not os.path.exists(game_substitutions_npy + '.npz') or not os.path.exists(player_lineups_npy + '.npz'):

    game_line_ups = dict()
    game_substitutions = dict()
    if dataset_version == StatsBombDataset.BARCELONA:
        for i, season in enumerate(tqdm(seasons.itertuples())):
            season_games = games[games.season_id == season.season_id]
            season_players = json.loads(player_list_per_season[str(season.season_id)])['player_id']
            season_players = set(map(lambda x: int(player_id_to_index[x]), season_players.values()))
            for game in tqdm(season_games.itertuples()):
                game_line_up = np.zeros((max_minutes + 1, 3, 2, len(player_id_encoder.classes_)))
                game_substitution = np.zeros((max_minutes + 1, 3, 3))
                game_events = SBL.events(game.game_id)
                barca_events = game_events[game_events.team_id == barcelona_id]
                substitution_events = barca_events[barca_events.type_name == "Substitution"]
                subs_batches = get_substitution_batches(substitution_events, nb_sub_batches)
                starting_XI_event = barca_events[barca_events.type_id == starting_XI_type_id]
                players_on_field = set(map(lambda x: player_id_to_index[x['player']['id']],
                                           starting_XI_event.iloc[0].extra['tactics']['lineup']))
                players_on_bench = season_players - players_on_field
                game_line_up[:, :, player_on_bench_index, list(players_on_bench)] = 1
                game_line_up[:, :, player_on_field_index, list(players_on_field)] = 1
                for sub_batch_nb in range(nb_sub_batches):
                    sub_batch = subs_batches[sub_batch_nb]
                    for substitution_event in sub_batch.itertuples():
                        sub_time = substitution_event.minute
                        game_substitution[sub_time][sub_batch_nb][0] = 1
                        player_out = player_id_to_index[substitution_event.player_id]
                        player_in = player_id_to_index[substitution_event.extra['substitution']['replacement']['id']]
                        game_substitution[sub_time][sub_batch_nb][player_in_index + 1] = player_in
                        game_substitution[sub_time][sub_batch_nb][player_out_index + 1] = player_out
                        players_on_field.add(player_in)
                        players_on_field.remove(player_out)
                        players_on_bench.add(player_out)
                        players_on_bench.remove(player_in)
                        game_line_up[sub_time + 1:, sub_batch_nb:, player_on_bench_index, list(players_on_bench)] = 1
                        game_line_up[sub_time + 1:, sub_batch_nb:, player_on_bench_index, list(players_on_field)] = 0
                        game_line_up[sub_time + 1:, sub_batch_nb:, player_on_field_index, list(players_on_field)] = 1
                        game_line_up[sub_time + 1:, sub_batch_nb:, player_on_field_index, list(players_on_bench)] = 0
                # game_substitution = pd.DataFrame(game_substitution, columns=['substitution', 'player_in', 'player_out'])
                # game_substitution.to_hdf(player_substitutions_h5, f'substitutions/game_{game.game_id}')
                game_substitutions[str(game.game_id)] = game_substitution
                game_line_ups[str(game.game_id)] = game_line_up
        np.savez(game_substitutions_npy, **game_substitutions)
        np.savez(player_lineups_npy, **game_line_ups)
        game_substitutions = np.load(game_substitutions_npy + '.npz')
        game_line_ups = np.load(player_lineups_npy + '.npz')
    else:
        for i, season in enumerate(tqdm(seasons.itertuples())):
            season_games = games[games.season_id == season.season_id]
            # season_games = season_games[games.game_id == 3775614]
            for game in tqdm(season_games.itertuples()):
                home_season_players = json.loads(player_list_per_season[str(season.season_id)][str(game.home_team_id)])[
                    'player_id']
                home_season_players = set(map(lambda x: int(player_id_to_index[x]), home_season_players.values()))
                away_season_players = json.loads(player_list_per_season[str(season.season_id)][str(game.away_team_id)])[
                    'player_id']
                away_season_players = set(map(lambda x: int(player_id_to_index[x]), away_season_players.values()))
                home_game_line_up = np.zeros((max_minutes + 1, nb_sub_batches, 2, len(player_id_encoder.classes_)))
                away_game_line_up = np.zeros((max_minutes + 1, nb_sub_batches, 2, len(player_id_encoder.classes_)))
                home_game_substitution = np.zeros((max_minutes + 1, nb_sub_batches, 3))
                away_game_substitution = np.zeros((max_minutes + 1, nb_sub_batches, 3))

                game_events = SBL.events(game.game_id)
                home_events = game_events[game_events.team_id == game.home_team_id]
                away_events = game_events[game_events.team_id == game.away_team_id]
                home_starting_XI_event = home_events[home_events.type_id == starting_XI_type_id]
                away_starting_XI_event = away_events[away_events.type_id == starting_XI_type_id]
                home_sub_events = home_events[home_events.type_name == "Substitution"]
                away_sub_events = away_events[away_events.type_name == "Substitution"]
                home_subs_batches = get_substitution_batches(home_sub_events, nb_sub_batches)
                away_subs_batches = get_substitution_batches(away_sub_events, nb_sub_batches)
                # print(game_events[game_events.type_name == "Substitution"][['team_id', 'extra', 'minute']])
                # print(home_sub_events)
                # print(away_sub_events)
                # print(gfd)
                home_players_on_field = set(map(lambda x: player_id_to_index[x['player']['id']],
                                                home_starting_XI_event.iloc[0].extra['tactics']['lineup']))
                away_players_on_field = set(map(lambda x: player_id_to_index[x['player']['id']],
                                                away_starting_XI_event.iloc[0].extra['tactics']['lineup']))
                home_players_on_bench = home_season_players - home_players_on_field
                away_players_on_bench = away_season_players - away_players_on_field
                home_game_line_up[:, :, player_on_bench_index, list(home_players_on_bench)] = 1
                home_game_line_up[:, :, player_on_field_index, list(home_players_on_field)] = 1
                away_game_line_up[:, :, player_on_bench_index, list(away_players_on_bench)] = 1
                away_game_line_up[:, :, player_on_field_index, list(away_players_on_field)] = 1
                for sub_batch_nb in range(nb_sub_batches):
                    home_sub_batch = home_subs_batches[sub_batch_nb]
                    for substitution_event in home_sub_batch.itertuples():
                        sub_time = substitution_event.minute
                        home_game_substitution[sub_time][sub_batch_nb][0] = 1
                        player_out = player_id_to_index[substitution_event.player_id]
                        player_in = player_id_to_index[substitution_event.extra['substitution']['replacement']['id']]
                        home_game_substitution[sub_time][sub_batch_nb][player_in_index + 1] = player_in
                        home_game_substitution[sub_time][sub_batch_nb][player_out_index + 1] = player_out
                        home_players_on_field.add(player_in)
                        home_players_on_field.remove(player_out)
                        home_players_on_bench.add(player_out)
                        home_players_on_bench.remove(player_in)
                        home_game_line_up[sub_time + 1:, sub_batch_nb:, player_on_bench_index,
                        list(home_players_on_bench)] = 1
                        home_game_line_up[sub_time + 1:, sub_batch_nb:, player_on_bench_index,
                        list(home_players_on_field)] = 0
                        home_game_line_up[sub_time + 1:, sub_batch_nb:, player_on_field_index,
                        list(home_players_on_field)] = 1
                        home_game_line_up[sub_time + 1:, sub_batch_nb:, player_on_field_index,
                        list(home_players_on_bench)] = 0
                for sub_batch_nb in range(nb_sub_batches):
                    away_sub_batch = away_subs_batches[sub_batch_nb]
                    for substitution_event in away_sub_batch.itertuples():
                        sub_time = substitution_event.minute
                        away_game_substitution[sub_time][sub_batch_nb][0] = 1
                        player_out = player_id_to_index[substitution_event.player_id]
                        player_in = player_id_to_index[substitution_event.extra['substitution']['replacement']['id']]
                        away_game_substitution[sub_time][sub_batch_nb][player_in_index + 1] = player_in
                        away_game_substitution[sub_time][sub_batch_nb][player_out_index + 1] = player_out
                        away_players_on_field.add(player_in)
                        away_players_on_field.remove(player_out)
                        away_players_on_bench.add(player_out)
                        away_players_on_bench.remove(player_in)
                        away_game_line_up[sub_time + 1:, sub_batch_nb:, player_on_bench_index,
                        list(away_players_on_bench)] = 1
                        away_game_line_up[sub_time + 1:, sub_batch_nb:, player_on_bench_index,
                        list(away_players_on_field)] = 0
                        away_game_line_up[sub_time + 1:, sub_batch_nb:, player_on_field_index,
                        list(away_players_on_field)] = 1
                        away_game_line_up[sub_time + 1:, sub_batch_nb:, player_on_field_index,
                        list(away_players_on_bench)] = 0
                # home_game_substitution = pd.DataFrame(home_game_substitution,
                #                                       columns=['substitution', 'player_in', 'player_out'])
                # print(home_game_substitution.iloc[58])
                # home_game_substitution.to_hdf(player_substitutions_h5, f'home_substitutions/game_{game.game_id}')
                game_substitutions[f'{game.game_id}/home'] = home_game_substitution
                game_line_ups[f'{game.game_id}/home'] = home_game_line_up
                # away_game_substitution = pd.DataFrame(away_game_substitution,
                #                                       columns=['substitution', 'player_in', 'player_out'])
                # print(away_game_substitution.iloc[70])
                # print(gdfg)
                # away_game_substitution.to_hdf(player_substitutions_h5, f'away_substitutions/game_{game.game_id}')
                game_substitutions[f'{game.game_id}/away'] = away_game_substitution
                game_line_ups[f'{game.game_id}/away'] = away_game_line_up

        np.savez(player_lineups_npy, **game_line_ups)
        np.savez(game_substitutions_npy, **game_substitutions)

        game_line_ups = np.load(player_lineups_npy + '.npz')
        game_substitutions = np.load(game_substitutions_npy + '.npz')

In [ ]:
game_list = []
for season in seasons.itertuples():
    season_games = games[games.season_id == season.season_id]
    sorted_season_games = season_games.sort_values('game_day', ascending=True)
    for game in tqdm(sorted_season_games.itertuples()):
        game_list.append(int(game.game_id))
game_list = np.tile(game_list, 2)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]